In [3]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from delta.assets import download_assets
from delta.utilities import cfg
from delta.model import unet_seg
from delta.data import trainGenerator_seg, seg_weights

import pathlib
import json
import h5py

import numpy as np
from numpy.random import default_rng
import tifffile

In [4]:
def to_str(posixpath):
    return str(posixpath.resolve())  

In [8]:
#root = pathlib.Path(pathlib.Path.home(), 'home', 'Caulobacter')

root = pathlib.Path(pathlib.Path.home(), 'Andreas', 'DeltaTest')
data_dir = root / 'data'
model_dir = root / 'models'

training_file = '/Users/simonvanvliet/Andreas/Training/training_data.hdf5'

validation_frac = 0.2

training_set = data_dir / 'training'
validation_set = data_dir / 'validation'
savefile = model_dir / 'Caulobacter2D'

In [12]:
download_assets = False
setup_processing = True

#make folders
data_dir.mkdir(exist_ok=True)
model_dir.mkdir(exist_ok=True)
training_set.mkdir(exist_ok=True)
validation_set.mkdir(exist_ok=True)

(training_set / 'img').mkdir(exist_ok=True)
(training_set / 'seg').mkdir(exist_ok=True)
(training_set / 'wei').mkdir(exist_ok=True)

(validation_set / 'img').mkdir(exist_ok=True)
(validation_set / 'seg').mkdir(exist_ok=True)
(validation_set / 'wei').mkdir(exist_ok=True)
    
if download_assets:
    #download config and default model weights
    download_assets(
        load_models=to_str(model_dir),
        load_sets=False,
        load_evals=False,
        config_level=to_str(root)
    )

if setup_processing:    
    # modify config file
    config_filename = root / 'config_2D.json'
    with open(config_filename) as f:
        config = json.load(f)
        
    #point to model location
    config['model_file_seg'] = to_str(model_dir / 'unet_pads_seg.hdf5')    
    config['model_file_track'] = to_str(model_dir / 'unet_pads_track.hdf5')    

    #only needed for model training, in that case, point to location of training data
    config['training_set_seg'] = to_str(data_dir / 'training')
    config['training_set_track'] = ''

    #point to raw data location
    config['eval_movie'] = to_str(data_dir / 'evaluation')

    #specify output formats
    config['save_format'] = ["pickle", "movie"]    


    new_config_filename = root / 'config_2D_caulobacter.json'
    with open(new_config_filename, 'w') as f:
        json.dump(config, f, indent=2)   
        
        
    #process training data
    rng = default_rng()

    with h5py.File(training_file, 'r') as f:
        labels = np.array(f['label'])
        images = np.array(f['images'])
      
    n_im = labels.shape[0]
    n_val = int(np.ceil(n_im * validation_frac))  
    
    val_set = rng.choice(n_im, size=n_val, replace=False)
    train_set = np.setdiff1d(np.arange(n_im), val_set, assume_unique=True)
    
    for idx, v in enumerate(val_set):
        
        im_name = validation_set / 'img' / ('img_%04i' % idx)
        lab_name = validation_set / 'seg' / ('img_%04i' % idx)
        wei_name = validation_set / 'wei' / ('img_%04i' % idx)
        
        c_lab = np.squeeze(labels[v,:,:]) > 0 
        c_lab.astype(np.uint8)
        
        tifffile.imwrite(im_name, np.squeeze(images[v,:,:]))
        tifffile.imwrite(lab_name, c_lab)
        tifffile.imwrite(wei_name, seg_weights(c_lab))
        
        
    for idx, v in enumerate(train_set):
        
        im_name = training_set / 'img' / ('img_%04i' % idx)
        lab_name = training_set / 'seg' / ('img_%04i' % idx)
        wei_name = training_set / 'wei' / ('img_%04i' % idx)
        
        c_lab = np.squeeze(labels[v,:,:]) > 0 
        c_lab.astype(np.uint8)
        
        tifffile.imwrite(im_name, np.squeeze(images[v,:,:]))
        tifffile.imwrite(lab_name, c_lab)
        tifffile.imwrite(wei_name, seg_weights(c_lab))    
             

KeyboardInterrupt: 

In [2]:
'img_%03i' % 3

'img_003'

In [ ]:
# Load config:
cfg.load_config(new_config_filename)

# Training parameters:
batch_size = 4
epochs = 600
steps_per_epoch = 200
patience = 50

#Data generator parameters:
data_gen_args = dict(
    rotation = 2,
    rotations_90d = True,
    zoom=.15,
    horizontal_flip=True,
    vertical_flip=True,
    histogram_voodoo=True,
    illumination_voodoo=True,
    gaussian_noise = 0.03,
    gaussian_blur = 1
    )

data_gen_args_val = dict(
    rotation = 2,
    rotations_90d = True,
    horizontal_flip=True,
    vertical_flip=True,
    histogram_voodoo=True,
    )

# Generator init:
gen_train = trainGenerator_seg(
    batch_size,
    os.path.join(to_str(training_set),'img'),
    os.path.join(to_str(training_set),'seg'),
    os.path.join(to_str(training_set),'wei'),
    augment_params = data_gen_args,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

gen_validate = trainGenerator_seg(
    batch_size,
    os.path.join(validation_set,'img'),
    os.path.join(validation_set,'seg'),
    os.path.join(validation_set,'wei'),
    augment_params = data_gen_args_val,
    target_size = cfg.target_size_seg,
    crop_windows = cfg.crop_windows
    )

# Define model:
model = unet_seg(input_size = cfg.target_size_seg+(1,), pretrained_weights=cfg.model_file_seg)
model.summary()

In [ ]:
# Callbacks:
model_checkpoint = ModelCheckpoint(
    to_str(savefile), monitor='loss', verbose=2, save_best_only=True
    )
early_stopping = EarlyStopping(
    monitor='loss', mode='min', verbose=2, patience=patience
    )
tensor_board = TensorBoard(log_dir="logs/cb")

In [ ]:
# Train:
history = model.fit(
    gen_train,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=gen_validate,
    validation_steps=5,
    callbacks=[model_checkpoint, early_stopping, tensor_board]
    )